In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API accesses to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage == 1.43.0
!pip install -q graphframes

In [ ]:
# general
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'information_retrieval_project'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name != 'pageviews-202108-user.pkl' and b.name != 'run_frontend_in_gcp.sh' and b.name != 'startup_script_gcp.sh':
        paths.append(full_path+b.name)

parquetFile = spark.read.parquet(*paths)

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0, SparkFiles.getRootDirectory())

# Page Views

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path)
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB)
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly
# total number of page views (5). Then, remove lines with article id or page
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# Create a Counter (dictionary) that sums up the pages views for the same
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
    for line in f:
        parts = line.split(' ')
        wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
    pickle.dump(wid2pv, f)

# Page Rank

In [ ]:
def generate_graph(pages):
  """ Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vertex (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  """
  # YOUR CODE HERE
  edges_rdd = pages.mapValues(lambda x: [t[0] for t in x]).flatMap(lambda x: [(x[0], v) for v in x[1]]).distinct()
  vertices_rdd = edges.flatMap(lambda x: [(v,) for v in x]).distinct()
  return edges_rdd, vertices_rdd

In [ ]:
pages_links = spark.read.parquet("gs://wikidata20210801_preprocess/*").select("id", "anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")

# Tokenization

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    Parameters:
    -----------
        text: string , representing the text to tokenize.
    Returns:
    -----------
        list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if
                      token.group() not in all_stopwords]
    return list_of_tokens

# Counts: Term Weights

In [ ]:
# PLACE YOUR CODE HERE
def word_count(text, id):
    """ Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs.
    """
    # YOUR CODE HERE
    word_counter = Counter()
    word_counter.update(tokenize(text))
    return [(token[0], (id, token[1])) for token in word_counter.items()]

def reduce_word_counts(unsorted_pl):
    """ Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    """
    # YOUR CODE HERE
    return sorted(unsorted_pl)

def get_doc_counter(text):
  """
  Calculates and returns a word counter for a given document
  """
  tokens = tokenize(text)
  word_counter = Counter(tokens)
  return word_counter

def calculate_df(postings):
    """ Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    """
    # YOUR CODE HERE
    return postings.map(lambda x: (x[0], len(x[1])))

def get_doc_length(text, id):
    """ Calculates the document's length, not including `all_stopwords`.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      tuple:
        (ID, Document length)
    """
    return id, len(tokenize(text))

def partition_postings_and_write(postings, index):
    """ A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    """
    # YOUR CODE HERE
    return postings.groupBy(lambda x: token2bucket_id(x[0])).map(lambda x: index.write_a_posting_list(x, bucket_name))

def createIndex(index, w2df_dict, totals_for_terms, doc_id_to_norm, DL_dict, avgDL, directory, isTitle):
    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=f'postings_gcp/'):
      if not blob.name.endswith("pickle"):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)

    # Adding the posting locations dictionary to the inverted index
    index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    index.df = w2df_dict
    # Add total number of terms dict
    index.totals_for_terms = totals_for_terms
    # Add DL_body dict
    index.DL = DL_dict
    # Add Average doc length
    index.avgDL = avgDL
    # Add number of terms
    index.number_of_terms = len(index.totals_for_terms)
    # Add norm
    index.doc_id_to_norm = doc_id_to_norm
    # Save id to title
    if isTitle:
        index.id_to_title = doc_title_pairs.collectAsMap()
    # write the global stats out
    index.write_index('.', directory)
    # upload to gs
    index_src = f'{directory}.pkl'
    index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
    !gsutil cp $index_src $index_dst
    !gsutil ls -lh $index_dst

# Create the Body Indexes

In [ ]:
from inverted_index_gcp import InvertedIndex as bodyIndex

body_index = bodyIndex()

# take the 'text' and 'id' and create an RDD from it
doc_text_pairs = parquetFile.select("text", "id").rdd

# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
postings = postings.filter(lambda x: len(x[1])>50)

w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

DL_RDD = doc_text_pairs.map(lambda x: get_doc_length(x[0], x[1]))

avgDL = DL_RDD.map(lambda x: x[1]).mean()

DL = DL_RDD.collectAsMap()

totals_for_terms = (word_counts.map(lambda x: (x[0], x[1][1]))).reduceByKey(lambda x, y: x + y).collectAsMap()

N = len(DL)

doc_id_to_norm = doc_text_pairs.map(lambda x: (x[1], get_doc_counter(x[0]))).map(lambda x: (x[0], np.linalg.norm([(x[1][term] / DL[x[0]]) * math.log(N / w2df_dict.get(term, N), 10) for term in x[1]]))).collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings, body_index).collect()

createIndex(body_index, w2df_dict, totals_for_terms, doc_id_to_norm, DL, avgDL, 'body_index', False)

# Create the Title Indexes

In [ ]:
from inverted_index_gcp import InvertedIndex as titleIndex

title_index = titleIndex()

# take the 'title' and 'id' and create an RDD from it
doc_title_pairs = parquetFile.select("title", "id").rdd

# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

DL_RDD = doc_title_pairs.map(lambda x: get_doc_length(x[0], x[1]))

avgDL = DL_RDD.map(lambda x: x[1]).mean()

DL = DL_RDD.collectAsMap()

totals_for_terms = (word_counts.map(lambda x: (x[0], x[1][1]))).reduceByKey(lambda x, y: x + y).collectAsMap()

N = len(DL)

doc_id_to_norm = doc_title_pairs.map(lambda x: (x[1], get_doc_counter(x[0]))).map(lambda x: (x[0], np.linalg.norm([(x[1][term] / DL[x[0]]) * math.log(N / w2df_dict.get(term, N), 10) for term in x[1]]))).collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings, title_index).collect()

createIndex(title_index, w2df_dict, totals_for_terms, doc_id_to_norm, DL, avgDL, 'title_index', True)

# Create the Anchor Indexes

In [ ]:
from inverted_index_gcp import InvertedIndex as anchorIndex

anchor_index = anchorIndex()

# take the 'anchor' and create an RDD from it
doc_anchor = parquetFile.select("anchor_text").rdd
doc_anchor_pairs = parquetFile.select("id","anchor_text").rdd.flatMap(lambda x :x[1]).groupByKey().mapValues(list).map(lambda x : (x[0]," ".join([y for y in x[1]])))


# word counts map
word_counts = doc_anchor.flatMap(lambda x: x.anchor_text).distinct().map(lambda x: word_count(x.text, x.id)).flatMap(lambda x: x).distinct()
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

DL_RDD = doc_anchor_pairs.map(lambda x: get_doc_length(x[0], x[1]))

avgDL = DL_RDD.map(lambda x: x[1]).mean()

DL = DL_RDD.collectAsMap()

totals_for_terms = (word_counts.map(lambda x: (x[0], x[1][1]))).reduceByKey(lambda x, y: x + y).collectAsMap()

N = len(DL)

doc_id_to_norm = doc_anchor_pairs.map(lambda x: (x[0], get_doc_counter(x[1]))).filter(lambda x: DL[x[0]] != 0).map(lambda x: (x[0], np.linalg.norm([(x[1][term] / DL[x[0]]) * math.log(N / w2df_dict.get(term, N), 10) for term in x[1]]))).collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings, anchor_index).collect()

createIndex(anchor_index, w2df_dict, totals_for_terms, doc_id_to_norm, DL, avgDL, 'anchor_index', False)